## Extracting the Coordinates

In [2]:
import requests
import json

In [3]:
# Overpass API endpoint
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

In [4]:
# Example query: All cycling routes in Europe (bicycle routes)
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#
query = """
[out:json][timeout:900];
(
  relation["route"="bicycle"](49.9,-7.6,58.7,1.8);
);
out geom;
"""
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#

In [5]:
# Make the request
response = requests.get(url, params={'data': query})

In [6]:
# Check success
if response.status_code == 200:
    data = response.json()
    print("✅ Success:", len(data['elements']), "routes found")
else:
    print("❌ Error:", response.status_code)

✅ Success: 8318 routes found


In [7]:
# Save response to file
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("UK_cycle_routes.json", "w") as f:
    json.dump(data, f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [8]:
import pandas as pd

# Create empty list to store results
routes_summary = []

# Loop through each route in the dataset
for route in data['elements']:
    if route['type'] != 'relation':
        continue  # skip non-route elements

    coords = []
    for member in route.get('members', []):
        if 'geometry' in member:
            segment_coords = [(pt['lon'], pt['lat']) for pt in member['geometry']]
            coords.extend(segment_coords)

    if coords:
        start_coord = coords[0]
        end_coord = coords[-1]
        routes_summary.append({
            'id': route.get('id'),
            'name': route.get('tags', {}).get('name', 'Unnamed route'),
            'start_lon': start_coord[0],
            'start_lat': start_coord[1],
            'end_lon': end_coord[0],
            'end_lat': end_coord[1],
            'num_points': len(coords)
        })

In [9]:
# Convert results to DataFrame
routes_df = pd.DataFrame(routes_summary)

In [10]:
routes_df = routes_df.sort_values(by='num_points', ascending=False)

# Extracting the features UK First 2,000

In [12]:
AK = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjNiMWE1ZTE5NzdkNjRjZTFhMTcyODM2ZmU3YmYzYzRiIiwiaCI6Im11cm11cjY0In0="

In [14]:
import openrouteservice
import pandas as pd
import json
import time
import os

In [15]:
# Initialize client
client = openrouteservice.Client(key=AK)  # Replace AK with your ORS API key

In [17]:
results = []
save_interval = 10

In [18]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
save_path = "ors_route_results_UK3.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [ ]:
test_df = routes_df.iloc[4001:6000]

In [20]:
test_df

,id,name,start_lon,start_lat,end_lon,end_lat,num_points
2789,9757389,Unnamed route,0.439065,51.566203,0.401111,51.568753,78


In [21]:
for i, row in test_df.iterrows():
    start = (row['start_lon'], row['start_lat'])
    end = (row['end_lon'], row['end_lat'])

    try:
        route = client.directions(
            coordinates=[start, end],
            profile='cycling-regular',
            format='geojson',
            elevation=True,
            instructions=True,
            extra_info=['surface', 'waytype', 'waycategory', 'steepness'],
        )


        
        # --- SAVE RAW ORS RESPONSE FOR DEBUGGING ---
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        raw_dir = "raw_ors_responses_UK3"
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        os.makedirs(raw_dir, exist_ok=True)
        
        raw_filename = f"{raw_dir}/route_{row['id']}.json"
        
        with open(raw_filename, "w") as raw_file:
            json.dump(route, raw_file, indent=2)
        # -------------------------------------------


        props = route['features'][0]['properties']
        summary = props['summary']
        extras = props.get('extras', {})
        segments = props['segments']
        steps = segments[0]['steps']
        turn_steps = [s for s in steps if s['type'] in {0,1,2,3,4,5,6,7}]
        turns = len(turn_steps)
        steps = len(steps)

        results.append({
            'id': row['id'],
            'name': row['name'],
            'distance_m': summary.get('distance'),
            'duration_s': summary.get('duration'),
            'ascent_m': [props['ascent']],
            'descent_m': [props['descent']],
            'steps': steps,
            'turns': turns,
            'surface': extras.get('surface', {}).get('values', []),
            'waytype': extras.get('waytype', {}).get('values', []),
            'waycategory': extras.get('waycategory', {}).get('values', []),
            'steepness': extras.get('steepness', {}).get('values', []),
        })

        print(f"✅ Route {i+1} processed successfully")
        time.sleep(1.5)  # rate-limit protection

        # Save partial file
        if (i + 1) % save_interval == 0:
            with open(save_path, "w") as f:
                json.dump(results, f, indent=2)
            print(f"💾 Saved partial results after {i+1} routes")

    except Exception as e:
        print(f"❌ Error on route {row['id']}: {e}")
        continue

✅ Route 2790 processed successfully
💾 Saved partial results after 2790 routes


In [22]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
final_path = "ors_route_results_UK_3.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [23]:
with open(final_path, "w") as f:
    json.dump(results, f, indent=2)

In [24]:
print(f"\n🎉 Finished run — processed {len(results)} routes.")
print(f"📁 Full output saved to {final_path}")


🎉 Finished run — processed 1 routes.
📁 Full output saved to ors_route_results_UK_3.json


In [25]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("ors_route_results_UK_3.json", "r") as f:
    results = json.load(f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [26]:
# Convert to a DataFrame
df = pd.DataFrame(results)

In [27]:
# Display the first few rows
display(df.head(1))

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,9757389,Unnamed route,2811.0,562.2,[15.1],[21.1],6,4,"[[0, 54, 3]]","[[0, 54, 6]]","[[0, 54, 0]]","[[0, 54, 0]]"


In [28]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
df.to_csv("UK3.csv", index=False)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [29]:
df

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,9757389,Unnamed route,2811.0,562.2,[15.1],[21.1],6,4,"[[0, 54, 3]]","[[0, 54, 6]]","[[0, 54, 0]]","[[0, 54, 0]]"
